# 시퀀스-투-시퀀스(Sequence-to-Sequence, seq2seq)

### 글자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기

다운로드 링크 : http://www.manythings.org/anki  
한국 코퍼스 데이터의 양이 작으므로 프랑스-영어 병렬 코퍼스인 fra-eng.zip 파일을 사용권장. 위의 링크에서 해당 파일을 다운받으시면 됩니다.

#### 1) 병렬 코퍼스 데이터에 대한 이해와 전처리

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [2]:
lines = pd.read_csv('C:/Users/it/Downloads/dataset/kor-eng/kor.txt', names=['src', 'tar', 'att'], sep='\t')

In [3]:
len(lines)

904

In [4]:
lines.head(10)

,src,tar,att
0,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hello!,안녕!,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
2,No way!,절대 아니야.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,No way!,그럴리가!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Goodbye!,안녕!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
5,I'm sad.,슬퍼.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
6,"Me, too.",나도.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
7,Perfect!,완벽해!,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
8,Shut up!,시끄러워!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
9,Welcome.,어서오세요.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [5]:
del lines["att"]

In [6]:
lines.sample(10)

,src,tar
360,I hate myself sometimes.,난 가끔 내 자신이 싫어.
822,Certain religions are against organ donation.,어떤 종교는 장기 기증을 금지하기도 해.
154,They're amateurs.,걔네 초짜야.
86,I love my home.,난 내 집이 좋아.
708,I'd like to reserve a table for two.,두 명 자리를 예약하고 싶어요.
731,When does your winter vacation begin?,겨울 방학은 언제 시작하나요?
882,Ebola spreads from person to person through bo...,에볼라는 체액을 통하여 사람에서 사람으로 전파된다.
748,Everybody here except me has done that.,나 빼고 여기에 있는 사람 모두 그것을 했다. (한 적이 있다.)
449,Tom went to bed very late.,톰은 엄청 늦게 잤어.
455,You'd better not tell him.,그에게 말하지 않는게 좋을걸.


In [7]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
820,Tom's birthday was the day before yesterday.,\t 톰의 생일은 그저께였다. \n
438,It's popular in Australia.,\t 호주에서는 인기가 있어요. \n
436,I took a walk with my dog.,\t 내 개와 산책했어. \n
266,I have money for you.,\t 너에게 줄 돈이 있다. \n
659,He was at the bottom of the class.,\t 걔는 반에서 꼴찌였어. \n
454,You will know soon enough.,\t 곧 충분히 알게 될거야. \n
515,Tom has high blood pressure.,\t 톰은 고혈압이다. \n
685,"Are you Swedish? ""No, I'm Swiss.""","\t 스웨덴 사람이세요? ""아니요, 스위스 사람이예요."" \n"
758,Tom and his brother look quite similar.,\t 톰이랑 톰네 형은 진짜 닮았어. \n
752,I have to introduce Tom to the manager.,\t 나는 톰을 매니저에게 소개시켜줘야 합니다. \n


In [8]:
# 글자 집합 구축
src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [9]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print(src_vocab_size)
print(tar_vocab_size)

70
662


In [10]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['d', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'а']
['개', '걀', '걔', '거', '걱', '건', '걸', '검', '겁', '것', '게', '겠', '겨', '격', '견', '결', '겼', '경', '계', '고', '곤', '곧', '곱', '곳', '공', '과', '관', '괜', '괴', '교']


In [11]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, "'": 4, ',': 5, '-': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '8': 15, '9': 16, ':': 17, ';': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'K': 30, 'L': 31, 'M': 32, 'N': 33, 'O': 34, 'P': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68, 'а': 69}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '8': 18, '9': 19, '?': 20, 'A': 21, 'B': 22, 'H': 23, 'M': 24, 'T': 25, 'a': 26, 'd': 27, 'h': 28, 'i': 29, 'm': 30, 'o': 31, 'p': 32, 'r': 33, 't': 34, 'y': 35, '가': 36, '각': 37, '간': 38, '갈': 39, '감': 40, '갑': 41, '값': 42, '갔': 43, '강': 4

In [12]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[41, 50, 57, 19], [27, 47, 54, 54, 57, 2], [33, 57, 1, 65, 43, 67, 2], [33, 57, 1, 65, 43, 67, 2], [26, 57, 57, 46, 44, 67, 47, 2]]


In [13]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 147, 76, 20, 3, 2], [1, 3, 395, 139, 4, 3, 2], [1, 3, 488, 167, 3, 394, 156, 405, 10, 3, 2], [1, 3, 83, 225, 249, 36, 4, 3, 2], [1, 3, 395, 139, 4, 3, 2]]


In [14]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 147, 76, 20, 3, 2], [3, 395, 139, 4, 3, 2], [3, 488, 167, 3, 394, 156, 405, 10, 3, 2], [3, 83, 225, 249, 36, 4, 3, 2], [3, 395, 139, 4, 3, 2]]


In [15]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

85
52


In [16]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [17]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

#### 3) seq2seq 기계 번역기 훈련시키기

In [18]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [19]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [20]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Train on 723 samples, validate on 181 samples
Epoch 1/50
723/723 [==============================] - 20s 28ms/sample - loss: 2.9279 - val_loss: 2.7356
Epoch 2/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.4308 - val_loss: 2.5778
Epoch 3/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.3924 - val_loss: 2.5688
Epoch 4/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.3852 - val_loss: 2.9253
Epoch 5/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.3625 - val_loss: 2.5718
Epoch 6/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.3466 - val_loss: 2.7968
Epoch 7/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.3396 - val_loss: 3.1174
Epoch 8/50
723/723 [==============================] - 14s 20ms/sample - loss: 1.3179 - val_loss: 2.8919
Epoch 9/50
723/723 [==============================] - 15s 20ms/sample - loss: 1.3047 - val_loss: 2.7352
Epoch 10/50
723/72

#### 4) seq2seq 기계 번역기 동작시키기

전체적인 번역 동작 단계를 정리하면 아래와 같습니다.
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
2. 상태와 <SOS>에 해당하는 '\t'를 디코더로 보냅니다.
3. 디코더가 <EOS>에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

In [21]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [22]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태를 이전 상태로 사용
decoder_states = [state_h, state_c]
# 이번에는 훈련 과정에서와 달리 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [23]:
index_to_src = dict(
    (i, char) for char, i in src_to_index.items())
index_to_tar = dict(
    (i, char) for char, i in tar_to_index.items())

In [24]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: #stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <sos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트 합니다.
        states_value = [h, c]

    return decoded_sentence

In [25]:
import numpy as np
for seq_index in [3,10,20,30,100]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: No way!
정답 문장:  그럴리가! 
번역기가 번역한 문장:  톰은 내 가 가 있어. 
-----------------------------------
입력 문장: Welcome.
정답 문장:  환영합니다. 
번역기가 번역한 문장:  톰은 내 가 가 있어. 
-----------------------------------
입력 문장: I'm sorry.
정답 문장:  죄송합니다. 
번역기가 번역한 문장:  톰은 그 가 가 있어. 
-----------------------------------
입력 문장: I felt bad.
정답 문장:  난 기분이 나빴다. 
번역기가 번역한 문장:  톰은 그 가 가 있어. 
-----------------------------------
입력 문장: Do you like rap?
정답 문장:  랩 좋아해? 
번역기가 번역한 문장:  톰은 내 가 가 있어. 
